In [1]:
import pandas as pd
import numpy as np

In [7]:
game_df = pd.read_csv("raw_data/Twitch_game_data.csv")
global_df = pd.read_csv("raw_data/Twitch_global_data.csv")
streamers_df = pd.read_csv("raw_data/twitchdata-update.csv")

In [16]:
Viewers = global_df[["year","Month","Avg_viewers"]]

In [18]:
Viewers.to_csv("data/global_viewers.csv")

In [ ]:
global_df